In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')
import pandas as pd
import spacy
import ru2
from IPython.display import display
pd.set_option('display.max_colwidth', 200)

In [3]:
pbool = lambda x: '+' if x else '-'
def pipe_add(n, names=['sentencizer']):
    pos = 0
    for x in names:
        if not x in n.pipe_names:
            if pos >= len(n.pipe_names):
                n.add_pipe(n.create_pipe(x))
                pos += 1
            else:
                n.add_pipe(n.create_pipe(x), before=n.pipe_names[pos])
                pos += 1
    return n

nlp_ru=pipe_add(spacy.load('../ru2'), ['sentencizer', 'tagger', 'parser'])
print("RU pipeline: {}".format(nlp_ru.pipe_names))

RU pipeline: ['sentencizer', 'tagger', 'parser', 'ner']


In [4]:
def parse(nlp, text):
    pbool = lambda x: '+' if x else '-'
    doc = nlp(text)
    sents=pd.DataFrame(data=[(str(s),) for s in doc.sents], columns=['Sentence']);
    print('NLP={}.{} Text={}'.format(nlp.__class__.__module__, nlp.__class__.__name__, doc))
    display(sents.head())
    toks=pd.DataFrame(data=[(t.shape_, pbool(t.orth_ in nlp.vocab), t.pos_,
                             t.text, t.tag_, t.lemma_, t.dep_, t.head) for t in doc],
                     columns=['shape', 'vocab', 'POS', 'text', 'tag', 'lemma', 'dep', 'head']);
    display(toks)
    #from spacy import displacy
    #displacy.serve(doc, style='dep')
text = 'Она дочь человека из соседней галактики . Славим дочь человека ! ' # + 'Карамамба !' 
#text = 'Налоги пропитаны потом всякого, кто трудится .'
parse(nlp_ru, text)


NLP=spacy.lang.ru.Russian Text=Она дочь человека из соседней галактики . Славим дочь человека ! 


,Sentence
0,Она дочь человека из соседней галактики .
1,Славим дочь человека !


,shape,vocab,POS,text,tag,lemma,dep,head
0,Xxx,+,PRON,Она,PRON__Case=Nom|Gender=Fem|Number=Sing|Person=3,она,nsubj,дочь
1,xxxx,+,NOUN,дочь,NOUN__Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,дочь,ROOT,дочь
2,xxxx,+,NOUN,человека,NOUN__Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing,человек,obj,дочь
3,xx,+,ADP,из,ADP___,из,case,галактики
4,xxxx,+,ADJ,соседней,ADJ__Case=Gen|Degree=Pos|Gender=Fem|Number=Sing,соседний,amod,галактики
5,xxxx,+,NOUN,галактики,NOUN__Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing,галактика,obl,дочь
6,.,+,PUNCT,.,PUNCT___,.,punct,дочь
7,Xxxxx,-,VERB,Славим,VERB__Aspect=Perf|Mood=Ind|Number=Plur|Person=1|Tense=Fut|VerbForm=Fin|Voice=Act,славим,ROOT,Славим
8,xxxx,+,NOUN,дочь,NOUN__Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,дочь,obj,Славим
9,xxxx,+,NOUN,человека,NOUN__Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing,человек,nmod,дочь


In [5]:
ru2.lemmatizer.DEBUG = 1
nlp_r2=pipe_add(ru2.Russian2())
print("R2 pipeline: {}".format(nlp_r2.pipe_names))
parse(nlp_r2, text)

Set lemma for selected POS tag: 	 SPACE {'POS': 'SPACE'}
Set lemma for selected POS tag: 
 SPACE {'POS': 'SPACE'}
Set lemma for selected POS tag:   SPACE {'POS': 'SPACE'}
Set lemma for selected POS tag: \n SPACE {'POS': 'SPACE'}
Set lemma for selected POS tag: \t SPACE {'POS': 'SPACE'}
Set lemma for selected POS tag:   SPACE {'POS': 'SPACE'}
R2 pipeline: ['sentencizer']
NLP=ru2.Russian2 Text=Она дочь человека из соседней галактики . Славим дочь человека ! 
No tagger, so we do Lemmatizer.lookup: Она
No tagger, so we do Lemmatizer.lookup: дочь
No tagger, so we do Lemmatizer.lookup: человека
No tagger, so we do Lemmatizer.lookup: из
Found 2 lookup alternatives for: из
No tagger, so we do Lemmatizer.lookup: соседней
No tagger, so we do Lemmatizer.lookup: галактики
No tagger, so we do Lemmatizer.lookup: .
No tagger, so we do Lemmatizer.lookup: Славим
No tagger, so we do Lemmatizer.lookup: дочь
No tagger, so we do Lemmatizer.lookup: человека
No tagger, so we do Lemmatizer.lookup: !


,Sentence
0,Она дочь человека из соседней галактики .
1,Славим дочь человека !


,shape,vocab,POS,text,tag,lemma,dep,head
0,Xxx,+,,Она,,она,,Она
1,xxxx,+,,дочь,,дочь,,дочь
2,xxxx,+,,человека,,человек,,человека
3,xx,+,,из,,из,,из
4,xxxx,+,,соседней,,соседний,,соседней
5,xxxx,+,,галактики,,галактика,,галактики
6,.,+,,.,,.,,.
7,Xxxxx,+,,Славим,,славить,,Славим
8,xxxx,+,,дочь,,дочь,,дочь
9,xxxx,+,,человека,,человек,,человека


In [6]:
nlp_en=pipe_add(spacy.load('en'))
print("EN pipeline: {}".format(nlp_en.pipe_names))
parse(nlp_en, text)
nlp_xx=pipe_add(spacy.load('xx'))
print("XX pipeline: {}".format(nlp_xx.pipe_names))
parse(nlp_xx, text)

EN pipeline: ['sentencizer', 'tagger', 'parser', 'ner']
NLP=spacy.lang.en.English Text=Она дочь человека из соседней галактики . Славим дочь человека ! 
XX pipeline: ['sentencizer', 'ner']
NLP=spacy.lang.xx.MultiLanguage Text=Она дочь человека из соседней галактики . Славим дочь человека ! 


,Sentence
0,Она дочь человека из соседней галактики .
1,Славим дочь человека !


,shape,vocab,POS,text,tag,lemma,dep,head
0,Xxx,+,NOUN,Она,NN,она,amod,дочь
1,xxxx,+,NOUN,дочь,NNS,дочь,nsubj,человека
2,xxxx,+,VERB,человека,VBP,человека,ROOT,человека
3,xx,+,ADP,из,IN,из,intj,человека
4,xxxx,+,VERB,соседней,VBP,соседней,compound,галактики
5,xxxx,+,NOUN,галактики,NNS,галактики,dobj,человека
6,.,+,PUNCT,.,.,.,punct,человека
7,Xxxxx,+,NOUN,Славим,NN,славим,compound,дочь
8,xxxx,+,NOUN,дочь,NNS,дочь,nsubj,человека
9,xxxx,+,VERB,человека,VBP,человека,ROOT,человека


,Sentence
0,Она дочь человека из соседней галактики .
1,Славим дочь человека !


,shape,vocab,POS,text,tag,lemma,dep,head
0,Xxx,+,,Она,,Она,,Она
1,xxxx,+,,дочь,,дочь,,дочь
2,xxxx,+,,человека,,человека,,человека
3,xx,+,,из,,из,,из
4,xxxx,+,,соседней,,соседней,,соседней
5,xxxx,+,,галактики,,галактики,,галактики
6,.,+,,.,,.,,.
7,Xxxxx,+,,Славим,,Славим,,Славим
8,xxxx,+,,дочь,,дочь,,дочь
9,xxxx,+,,человека,,человека,,человека


In [7]:
def inspect_object(x):
    import inspect
    opts = dir(x)
    
    for p in opts:
        if not '__' in p and not (p+'_' in opts) and not p.startswith('is_') and not p.startswith('like_'):
            try:
                a = getattr(x, p)
            except Exception as e:
                continue
                a = 'Exception: {}'.format(e)
            if callable(a): # don't call callables to avoid the object state change
                #print('.{}()'.format(a))
                continue
#                 try:
#                     #a = a()
#                     p = p + '()'
#                 except:
#                     pass
            if inspect.isgenerator(a):
                p = p+'[:]'
                a = list(a)
            #if hasattr(a, '__iter__'): a = list(a)
            print('.{} = {}'.format(p, a))

In [8]:
inspect_object(nlp_ru.vocab['она'])

.cluster = 0
.flags = 12306
.has_vector = False
.lang_ = ru
.lower_ = она
.norm_ = она
.orth_ = она
.prefix_ = о
.prob = -20.0
.rank = 0
.sentiment = 0.0
.shape_ = xxx
.suffix_ = она
.text = она
.vocab = <spacy.vocab.Vocab object at 0x7fd3ab9d1b88>


In [9]:
vocab = sorted([n.orth_ for n in nlp_ru.vocab])
print(len(vocab), ':')
display(vocab[::len(vocab)//300])

108205 :


['\t',
 '165',
 '20-летнее',
 '45,5',
 ':)))',
 'Force',
 'Perseo',
 'disasters',
 'Адама',
 'Андерсон',
 'БДО',
 'Береги',
 'Бриджет',
 'Валяясь',
 'Виксинты',
 'Воронью',
 'Вышки',
 'Гершензон',
 'Гражданское',
 'Дворников',
 'Добраться',
 'Европу',
 'Завоеватель',
 'Знаменитым',
 'Ингеллау',
 'КОЛЫБЕЛЬНАЯ',
 'Касьяновым',
 'Колумба',
 'Красногорского',
 'Лакмусовая',
 'Лоренцо',
 'Малым',
 'Мессии',
 'Молчащий',
 'Надежной',
 'Недоступная',
 'Новотроицка',
 'Овчинского',
 'Основанием',
 'ПОЛИТИЧЕСКИХ',
 'Перекукует',
 'Повторяю',
 'Помощь',
 'Представьте',
 'Произошедшее',
 'РПГ-18',
 'Резервы',
 'Руговы',
 'Санкт-Морице',
 'Сестра',
 'Соберет',
 'Способностью',
 'Суздальцев',
 'Темирбаев',
 'Труд',
 'Унификация',
 'Февр',
 'Хансен',
 'Церемония',
 'Шамиль',
 'Экспертного',
 'Японский',
 'автономных',
 'аккаунт',
 'анализа',
 'аплодисментами',
 'ассоциации',
 'базирующаяся',
 'бегство',
 'белками',
 'бессмысленности',
 'благами',
 'богатейшего',
 'борта',
 'буддизмом',
 'бэкграунда'

In [10]:
from spacy.lang.ru.lemmatizer import RussianLemmatizer
import ru2.lemmatizer
lemmatizer1 = RussianLemmatizer()
lemmatizer2 = ru2.lemmatizer.RussianLemmatizer()
display(lemmatizer1(u'электричеством', 'NOUN'))
display(lemmatizer1(u'электричеством', 'X'))
display(lemmatizer2(u'электричеством', 'NOUN'))
display(lemmatizer2(u'электричеством', 'X'))

['электричество']

['электричеством']

Set lemma for selected POS tag: электричеством NOUN None
Set lemma for selected POS tag: электричеством X None


['электричество']

['электричеством']

In [13]:
inspect_object(nlp_ru.vocab[101])

.cluster = 0
.flags = 8582
.has_vector = False
.lang_ = ru
.lower_ = x
.norm_ = x
.orth_ = X
.prefix_ = X
.prob = -20.0
.rank = 0
.sentiment = 0.0
.shape_ = X
.suffix_ = X
.text = X
.vocab = <spacy.vocab.Vocab object at 0x7fd3ab9d1b88>
